In [58]:
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd

In [6]:
# 컬럼들 안의 이상 값들
na_values = ['$', '#VALUE!', '##', 'XNA', '@', '#', 'x', '&']
# float데이터 인데, object로 되어있는 이상 컬럼들
columns = ['Client_Income', 'Credit_Amount', 'Loan_Annuity', 'Score_Source_3', 'Population_Region_Relative', 'Age_Days', 'Employed_Days', 'Registration_Days', 'ID_Days']
# csv파일 불러오는 과정 na_values안에 포함된 것들은 결측치로 대체
data = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/Train_Dataset.csv', na_values=na_values, encoding='utf-8', engine='python')


In [38]:
data2 =data.drop(['ID','Own_House_Age', 'Score_Source_1', 'Social_Circle_Default', 'Client_Occupation', 'Score_Source_3', 'Type_Organization', 'Credit_Bureau', 'Mobile_Tag'], axis=1)

In [30]:
no_drop_numeri = ['Age_Days',
                  'Child_Count',
                  'Cleint_City_Rating',
                  'Client_Family_Members',
                  'Client_Income',
                  'Credit_Amount',
                  'Employed_Days',
                  'ID_Days',
                  'Loan_Annuity',
                  'Phone_Change',
                  'Population_Region_Relative',
                  'Registration_Days',
                  'Score_Source_2']


In [31]:
no_drop_cat = ['Accompany_Client',
               'Active_Loan',
               'Application_Process_Day',
               'Application_Process_Hour',
               'Bike_Owned',
               'Car_Owned',
               'Client_Contact_Work_Tag',
               'Client_Education',
               'Client_Gender',
               'Client_Housing_Type',
               'Client_Income_Type',
               'Client_Marital_Status',
               'Client_Permanent_Match_Tag',
               'Homephone_Tag',
               'House_Own',
               'Loan_Contract_Type',
               'Workphone_Working']


In [34]:
data2.groupby('Default')[no_drop_numeri].median()

,Age_Days,Child_Count,Cleint_City_Rating,Client_Family_Members,Client_Income,Credit_Amount,Employed_Days,ID_Days,Loan_Annuity,Phone_Change,Population_Region_Relative,Registration_Days,Score_Source_2
Default,,,,,,,,,,,,,
0,15848.0,0.0,2.0,2.0,14850.0,51856.2,2296.0,3284.0,2497.050,774.0,0.018850,4532.0,0.572589
1,14381.5,0.0,2.0,2.0,13500.0,49752.0,1449.0,2786.5,2528.325,586.0,0.018801,4067.0,0.443746


In [39]:
data2.isna().sum()

Client_Income                 3622
Car_Owned                     3581
Bike_Owned                    3624
Active_Loan                   3635
House_Own                     3661
Child_Count                   3638
Credit_Amount                 3637
Loan_Annuity                  4826
Accompany_Client              1758
Client_Income_Type            3701
Client_Education              3645
Client_Marital_Status         3473
Client_Gender                 2416
Loan_Contract_Type            3651
Client_Housing_Type           3687
Population_Region_Relative    4868
Age_Days                      3617
Employed_Days                 3666
Registration_Days             3631
ID_Days                       5985
Homephone_Tag                    0
Workphone_Working                0
Client_Family_Members         2410
Cleint_City_Rating            2409
Application_Process_Day       2428
Application_Process_Hour      3663
Client_Permanent_Match_Tag       0
Client_Contact_Work_Tag          0
Score_Source_2      

In [48]:
#그룹별 중앙값 대체
for col in no_drop_numeri :
    data2[col] = data2.groupby('Default')[col].transform(lambda x: x.fillna(x.median()))
#그룹별 최빈값 대체
for col in no_drop_cat :
    vals = data2.groupby(['Default'])[col] \
         .transform(lambda x: x.mode(dropna=False).iloc[0])
    data2[col] = data2[col].fillna(vals)



In [52]:
data2.isna().sum()

Client_Income                 0
Car_Owned                     0
Bike_Owned                    0
Active_Loan                   0
House_Own                     0
Child_Count                   0
Credit_Amount                 0
Loan_Annuity                  0
Accompany_Client              0
Client_Income_Type            0
Client_Education              0
Client_Marital_Status         0
Client_Gender                 0
Loan_Contract_Type            0
Client_Housing_Type           0
Population_Region_Relative    0
Age_Days                      0
Employed_Days                 0
Registration_Days             0
ID_Days                       0
Homephone_Tag                 0
Workphone_Working             0
Client_Family_Members         0
Cleint_City_Rating            0
Application_Process_Day       0
Application_Process_Hour      0
Client_Permanent_Match_Tag    0
Client_Contact_Work_Tag       0
Score_Source_2                0
Phone_Change                  0
Default                       0
dtype: i

In [51]:
standard_scaler = StandardScaler()
onehot_encoder = OneHotEncoder()
prep_processor = ColumnTransformer(
     [('standard',standard_scaler,no_drop_numeri),
     ('onehot',onehot_encoder, no_drop_cat),
      
     ],remainder='passthrough'
)


In [55]:
Y = data2['Default']
X = data2.drop('Default', axis=1)

In [59]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [62]:
len(X.columns)

30

In [60]:
prep_processor.fit(train_x)

ColumnTransformer(remainder='passthrough',
                  transformers=[('standard', StandardScaler(),
                                 ['Age_Days', 'Child_Count',
                                  'Cleint_City_Rating', 'Client_Family_Members',
                                  'Client_Income', 'Credit_Amount',
                                  'Employed_Days', 'ID_Days', 'Loan_Annuity',
                                  'Phone_Change', 'Population_Region_Relative',
                                  'Registration_Days', 'Score_Source_2']),
                                ('onehot', OneHotEncoder(),
                                 ['Accompany_Client', 'Active_Loan',
                                  'Application_Process_Day',
                                  'Application_Process_Hour', 'Bike_Owned',
                                  'Car_Owned', 'Client_Contact_Work_Tag',
                                  'Client_Education', 'Client_Gender',
                                  'Client_Housing_Type', 'Client_Income_Type',
                                  'Client_Marital_Status',
                                  'Client_Permanent_Match_Tag', 'Homephone_Tag',
                                  'House_Own', 'Loan_Contract_Type',
                                  'Workphone_Working'])])

In [65]:
transform_x_train = prep_processor.transform(train_x)
transform_x_test = prep_processor.transform(test_x)


In [81]:
transform_x_test.shape

(24372, 93)

In [108]:
from functools import partial
from keras.layers import LeakyReLU



In [80]:
checkpoint_path = "deep_training/model_cp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1,)
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=300)

In [109]:
neural_model = tf.keras.models.Sequential(
    [
        keras.layers.Dense(units=93, input_shape=(93,), activation=keras.layers.LeakyReLU(alpha=0.01),kernel_regularizer=tf.keras.regularizers.L2(0)),
        keras.layers.Dense(units=186,  activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=tf.keras.regularizers.L2(0)),
        keras.layers.Dense(units=186, activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=tf.keras.regularizers.L2(0.00)),
        keras.layers.Dropout(0.3),  #0.7 under fitting 
        keras.layers.Dense(units=1, activation='sigmoid'),
        

    ]
    ,name='predict'
)

In [110]:
neural_model.summary()

Model: "predict"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 93)                8742      
                                                                 
 dense_25 (Dense)            (None, 186)               17484     
                                                                 
 dense_26 (Dense)            (None, 186)               34782     
                                                                 
 dropout_6 (Dropout)         (None, 186)               0         
                                                                 
 dense_27 (Dense)            (None, 1)                 187       
                                                                 
Total params: 61,195
Trainable params: 61,195
Non-trainable params: 0
_________________________________________________________________


In [113]:
neural_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0),
metrics=['binary_accuracy','Recall','Precision' ] )

In [114]:
hist = neural_model.fit(transform_x_train, train_y.to_numpy(), epochs=100, callbacks=[cp_callback,earlystop_callback], validation_split=0.2)

Epoch 1/100


c:\Users\PaleBlue\semi_project\venv\lib\site-packages\keras\backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2427/2438 [============================>.] - ETA: 0s - loss: 55.2836 - binary_accuracy: 0.9175 - recall: 0.0056 - precision: 0.1477
Epoch 1: saving model to deep_training\model_cp.ckpt


INFO:tensorflow:Assets written to: deep_training\model_cp.ckpt\assets


INFO:tensorflow:Assets written to: deep_training\model_cp.ckpt\assets


...

2438/2438 [==============================] - 5s 2ms/step - loss: 1.1206 - binary_accuracy: 0.9175 - recall: 0.0027 - precision: 0.0833 - val_loss: 0.4003 - val_binary_accuracy: 0.9171 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 24/100
2421/2438 [============================>.] - ETA: 0s - loss: 0.4467 - binary_accuracy: 0.9178 - recall: 0.0029 - precision: 0.1017
Epoch 24: saving model to deep_training\model_cp.ckpt


INFO:tensorflow:Assets written to: deep_training\model_cp.ckpt\assets


INFO:tensorflow:Assets written to: deep_training\model_cp.ckpt\assets


2438/2438 [==============================] - 5s 2ms/step - loss: 0.4461 - binary_accuracy: 0.9179 - recall: 0.0029 - precision: 0.1017 - val_loss: 0.3878 - val_binary_accuracy: 0.9174 - val_recall: 0.0025 - val_precision: 0.8000
Epoch 25/100
1760/2438 [====================>.........] - ETA: 1s - loss: 0.3988 - binary_accuracy: 0.9186 - recall: 0.0033 - precision: 0.2143

KeyboardInterrupt: 